<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#!pip install MySQL-python

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import json
import os
import MySQLdb
import getpass
import random
import numpy as np
import re
import matplotlib.pyplot as plt

In [3]:
yr_start = 2017
yr_end = 2018
re_new = True
#----------batter or pitcher-----------
player_type = 'batter'

In [4]:
#-------------------YWL made password invisible-------------------
import getpass
ID = input('MySQL ID: ')
passwd = getpass.getpass('password: ')
#------------------YWL----------------------------------

MySQL ID: ywliao
password: ········


In [5]:
def data_type(str):
    try:
        if float(str) % 1 == 0:
            this_type = 'MEDIUMINT'
        else:
            this_type = 'FLOAT'
    except:
        this_type = 'CHAR(30)'
    return this_type

In [6]:
def get_pitcher_arsenal(yr, player_type):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    headers = {"User-Agent": user_agent}
    url = f'https://baseballsavant.mlb.com/leaderboard/pitch-arsenal-stats?type={player_type}&pitchType=&year={yr}&team=&min=1'
    
    req = requests.get(url=url, headers=headers)
    req.encoding = 'utf-8'
    soup = bs(req.text, 'html.parser')
    
    table = soup.find('div',{'class':'article-template'}).script
    table_element = str(table).split(',')

    for i, this_element in enumerate(table_element):
        if 'player_name' in this_element or 'href' in this_element:
            table_element[i] = this_element + ',' +table_element[i+1]
            table_element[i+1] = 'to_be_removed'
    table_element.remove('to_be_removed')
    table_element = table_element[4::]
    
    players_pitch_type = []
    player_name = False
    link = False
    this_dict = {}
    for this_TE in table_element:
        this_TE = this_TE.replace('}','')
        if this_TE.find('{') >= 0:
            this_TE = this_TE.split('{')[1]
        this_TE2 = this_TE.split(':')
        if (not this_TE[1:5] == 'href') and (len(this_TE2) == 2):
            this_key = this_TE2[0].replace('"','')
            this_value = this_TE2[1].replace('"','')
            this_dict[this_key] = this_value
            if this_key == 'rowId':
                players_pitch_type.append(this_dict)
                this_dict = {}
    
    key_list = list(players_pitch_type[0].keys())
    valid_count = np.zeros((len(key_list)))
    key_type = {}
    for player in players_pitch_type:
        for i, key in enumerate(key_list):
            try:
                if not player[key][0:4] == 'null':
                    valid_count[i] += 1
                    if key_type.__contains__(key):
                        if not key_type[key] == 'FLOAT':
                            key_type[key] = data_type(player[key])
                    else:
                        key_type[key] = data_type(player[key])
            except:
                pass

    valid_key = []
    for i, key in enumerate(key_list):
        if valid_count[i] > 0:
            valid_key.append(key)

    valid_key.remove('player_type')
    valid_key.remove('year')
    
    db = MySQLdb.connect("localhost", ID, passwd, "baseball_stats")
    cursor = db.cursor()
    table_name = str(yr)+player_type+'_pitcher_arsenal'
    sql = f"drop table if exists {table_name}"
    cursor.execute(sql)
    sql = f'create table {table_name} (rowId char(15) not null primary key) default charset = utf8'
    cursor.execute(sql)
    sql = f'ALTER TABLE {table_name} ADD COLUMN player_name char(30)'
    cursor.execute(sql)
    
    for i, key in enumerate(valid_key):
        if not (key == 'rowId' or key == 'player_name'):
            sql_tail = key + f' {key_type[key]}'
            sql = f'ALTER TABLE {table_name} ADD COLUMN ' + sql_tail
            cursor.execute(sql)
    
    player_num = len(players_pitch_type)
    
    
    for i, player in enumerate(players_pitch_type):
        sql = f'insert into {table_name}(rowId) values("{player["rowId"]}")'
        cursor.execute(sql)
        db.commit()
        for key in valid_key:
            if not key == 'rowId':
                if not player[key][0:4] == 'null':
                    value_str = player[key]
                    if key_type[key] == 'CHAR(30)':
                        value_str = '\"' + value_str + '\"'
                    elif value_str == '':
                        value_str = '0'
                        


                    try:
                        sql = f'update {table_name} set {key}={value_str} \
                        where rowId="{player["rowId"]}"'
                        #print(sql)
                        cursor.execute(sql)
                    except:
                        print(sql)
                        #sql2 = f'update {table_name} set {key}={player[key][0:5]} \
                        #where rowId="{player["rowId"]}"'
                        #cursor.execute(sql2)
        db.commit()
        print(f'Year = {yr}, {i+1}/{player_num}, {player["player_name"]}')
    db.close()
    return table_element

In [7]:
for player_type in ['batter','pitcher']:
    for yr in range(yr_start,yr_end+1):
        te = get_pitcher_arsenal(yr, player_type)

Year = 2017, 1/6015, Dozier, Brian
Year = 2017, 2/6015, Votto, Joey
Year = 2017, 3/6015, Blackmon, Charlie
Year = 2017, 4/6015, LeMahieu, DJ
Year = 2017, 5/6015, Hamilton, Billy
Year = 2017, 6/6015, Bryant, Kris
Year = 2017, 7/6015, Strange-Gordon, Dee
Year = 2017, 8/6015, Arenado, Nolan
Year = 2017, 9/6015, Carpenter, Matt
Year = 2017, 10/6015, Yelich, Christian
Year = 2017, 11/6015, Rizzo, Anthony
Year = 2017, 12/6015, Galvis, Freddy
Year = 2017, 13/6015, Seager, Corey
Year = 2017, 14/6015, Gardner, Brett
Year = 2017, 15/6015, Span, Denard
Year = 2017, 16/6015, Hernández, César
Year = 2017, 17/6015, Franco, Maikel
Year = 2017, 18/6015, Rendon, Anthony
Year = 2017, 19/6015, Posey, Buster
Year = 2017, 20/6015, Reynolds, Mark
Year = 2017, 21/6015, Mauer, Joe
Year = 2017, 22/6015, Lindor, Francisco
Year = 2017, 23/6015, Lowrie, Jed
Year = 2017, 24/6015, Inciarte, Ender
Year = 2017, 25/6015, Encarnación, Edwin
Year = 2017, 26/6015, Duvall, Adam
Year = 2017, 27/6015, Bell, Josh
Year = 2017

Year = 2017, 225/6015, Anderson, Tim
Year = 2017, 226/6015, Walker, Neil
Year = 2017, 227/6015, Báez, Javier
Year = 2017, 228/6015, Descalso, Daniel
Year = 2017, 229/6015, Margot, Manuel
Year = 2017, 230/6015, Upton, Justin
Year = 2017, 231/6015, Pollock, AJ
Year = 2017, 232/6015, Seager, Kyle
Year = 2017, 233/6015, Frazier, Adam
Year = 2017, 234/6015, Betts, Mookie
Year = 2017, 235/6015, Rizzo, Anthony
Year = 2017, 236/6015, Peralta, David
Year = 2017, 237/6015, McCutchen, Andrew
Year = 2017, 238/6015, Lowrie, Jed
Year = 2017, 239/6015, Contreras, Willson
Year = 2017, 240/6015, Votto, Joey
Year = 2017, 241/6015, DeJong, Paul
Year = 2017, 242/6015, Gonzalez, Marwin
Year = 2017, 243/6015, Canó, Robinson
Year = 2017, 244/6015, Judge, Aaron
Year = 2017, 245/6015, Zobrist, Ben
Year = 2017, 246/6015, Strange-Gordon, Dee
Year = 2017, 247/6015, Altherr, Aaron
Year = 2017, 248/6015, Cabrera, Asdrúbal
Year = 2017, 249/6015, Machado, Manny
Year = 2017, 250/6015, Bour, Justin
Year = 2017, 251/601

Year = 2017, 449/6015, Espinosa, Danny
Year = 2017, 450/6015, Lucroy, Jonathan
Year = 2017, 451/6015, Escobar, Alcides
Year = 2017, 452/6015, Tomlinson, Kelby
Year = 2017, 453/6015, Ozuna, Marcell
Year = 2017, 454/6015, Almora Jr., Albert
Year = 2017, 455/6015, Morales, Kendrys
Year = 2017, 456/6015, Mancini, Trey
Year = 2017, 457/6015, Báez, Javier
Year = 2017, 458/6015, Beckham, Tim
Year = 2017, 459/6015, Romine, Andrew
Year = 2017, 460/6015, Joseph, Tommy
Year = 2017, 461/6015, Buxton, Byron
Year = 2017, 462/6015, Bonifacio, Jorge
Year = 2017, 463/6015, Garcia, Greg
Year = 2017, 464/6015, Revere, Ben
Year = 2017, 465/6015, Drury, Brandon
Year = 2017, 466/6015, Chapman, Matt
Year = 2017, 467/6015, Sánchez, Yolmer
Year = 2017, 468/6015, Duvall, Adam
Year = 2017, 469/6015, Story, Trevor
Year = 2017, 470/6015, Arcia, Orlando
Year = 2017, 471/6015, Gómez, Carlos
Year = 2017, 472/6015, Gyorko, Jedd
Year = 2017, 473/6015, Smith, Mallex
Year = 2017, 474/6015, Rizzo, Anthony
Year = 2017, 475

Year = 2017, 672/6015, Choo, Shin-Soo
Year = 2017, 673/6015, Piña, Manny
Year = 2017, 674/6015, Thames, Eric
Year = 2017, 675/6015, Robertson, Daniel
Year = 2017, 676/6015, Bruce, Jay
Year = 2017, 677/6015, Galvis, Freddy
Year = 2017, 678/6015, Hanson, Alen
Year = 2017, 679/6015, Gattis, Evan
Year = 2017, 680/6015, Russell, Addison
Year = 2017, 681/6015, Hosmer, Eric
Year = 2017, 682/6015, Solarte, Yangervis
Year = 2017, 683/6015, Moustakas, Mike
Year = 2017, 684/6015, Carrera, Ezequiel
Year = 2017, 685/6015, González, Carlos
Year = 2017, 686/6015, Semien, Marcus
Year = 2017, 687/6015, León, Sandy
Year = 2017, 688/6015, Granderson, Curtis
Year = 2017, 689/6015, Smoak, Justin
Year = 2017, 690/6015, Suzuki, Ichiro
Year = 2017, 691/6015, Mazara, Nomar
Year = 2017, 692/6015, Russell, Addison
Year = 2017, 693/6015, Calhoun, Kole
Year = 2017, 694/6015, Harper, Bryce
Year = 2017, 695/6015, Gyorko, Jedd
Year = 2017, 696/6015, Peralta, David
Year = 2017, 697/6015, Ozuna, Marcell
Year = 2017, 69

Year = 2017, 895/6015, Schoop, Jonathan
Year = 2017, 896/6015, Calhoun, Kole
Year = 2017, 897/6015, Bogaerts, Xander
Year = 2017, 898/6015, Rizzo, Anthony
Year = 2017, 899/6015, Marte, Ketel
Year = 2017, 900/6015, Kendrick, Howie
Year = 2017, 901/6015, Headley, Chase
Year = 2017, 902/6015, Schwarber, Kyle
Year = 2017, 903/6015, Moustakas, Mike
Year = 2017, 904/6015, Morrison, Logan
Year = 2017, 905/6015, Rosales, Adam
Year = 2017, 906/6015, Joyce, Matt
Year = 2017, 907/6015, Mauer, Joe
Year = 2017, 908/6015, Braun, Ryan
Year = 2017, 909/6015, Kepler, Max
Year = 2017, 910/6015, Iannetta, Chris
Year = 2017, 911/6015, Yelich, Christian
Year = 2017, 912/6015, Cespedes, Yoenis
Year = 2017, 913/6015, Smith, Seth
Year = 2017, 914/6015, Wolters, Tony
Year = 2017, 915/6015, Kepler, Max
Year = 2017, 916/6015, Gonzalez, Marwin
Year = 2017, 917/6015, Herrera, Odúbel
Year = 2017, 918/6015, Duda, Lucas
Year = 2017, 919/6015, Margot, Manuel
Year = 2017, 920/6015, Escobar, Yunel
Year = 2017, 921/6015,

...

Year = 2018, 2059/3075, Peterson, Tim
Year = 2018, 2060/3075, Cortes, Nestor
Year = 2018, 2061/3075, Hauschild, Mike
Year = 2018, 2062/3075, Fields, Josh
Year = 2018, 2063/3075, Liriano, Francisco
Year = 2018, 2064/3075, Phillips, Evan
Year = 2018, 2065/3075, López, Reynaldo
Year = 2018, 2066/3075, Clippard, Tyler
Year = 2018, 2067/3075, Bowman, Matt
Year = 2018, 2068/3075, Ryu, Hyun Jin
Year = 2018, 2069/3075, Alexander, Scott
Year = 2018, 2070/3075, Ramirez, José
Year = 2018, 2071/3075, Ureña, José
Year = 2018, 2072/3075, Rodríguez, Ricardo
Year = 2018, 2073/3075, Schultz, Jaime
Year = 2018, 2074/3075, Wood, Blake
Year = 2018, 2075/3075, Jurado, Ariel
Year = 2018, 2076/3075, Baumann, Buddy
Year = 2018, 2077/3075, Tazawa, Junichi
Year = 2018, 2078/3075, Almonte, Miguel
Year = 2018, 2079/3075, Hudson, Dakota
Year = 2018, 2080/3075, Domínguez, Seranthony
Year = 2018, 2081/3075, Bergman, Christian
Year = 2018, 2082/3075, Britton, Zack
Year = 2018, 2083/3075, Feliz, Michael
Year = 2018, 2

Year = 2018, 2277/3075, Lockett, Walker
Year = 2018, 2278/3075, Albers, Matt
Year = 2018, 2279/3075, Kratz, Erik
Year = 2018, 2280/3075, Kelly, Casey
Year = 2018, 2281/3075, Shipley, Braden
Year = 2018, 2282/3075, López, Yoan
Year = 2018, 2283/3075, Paredes, Edward
Year = 2018, 2284/3075, Gonzalez, Merandy
Year = 2018, 2285/3075, O'Grady, Chris
Year = 2018, 2286/3075, Curtis, Zac
Year = 2018, 2287/3075, Shoemaker, Matt
Year = 2018, 2288/3075, Despaigne, Odrisamer
Year = 2018, 2289/3075, Hoffman, Jeff
Year = 2018, 2290/3075, Rusin, Chris
Year = 2018, 2291/3075, Gray, Jon
Year = 2018, 2292/3075, Detwiler, Ross
Year = 2018, 2293/3075, Bradford, Chasen
Year = 2018, 2294/3075, Morgan, Adam
Year = 2018, 2295/3075, Stephenson, Robert
Year = 2018, 2296/3075, Romano, Sal
Year = 2018, 2297/3075, Casilla, Santiago
Year = 2018, 2298/3075, Jackson, Edwin
Year = 2018, 2299/3075, Yarbrough, Ryan
Year = 2018, 2300/3075, Schultz, Jaime
Year = 2018, 2301/3075, Tomlin, Josh
Year = 2018, 2302/3075, Voth, 

Year = 2018, 2504/3075, Detwiler, Ross
Year = 2018, 2505/3075, Darvish, Yu
Year = 2018, 2506/3075, Wendelken, J.B.
Year = 2018, 2507/3075, Maton, Phil
Year = 2018, 2508/3075, Triggs, Andrew
Year = 2018, 2509/3075, Bracho, Silvino
Year = 2018, 2510/3075, Pelham, CD
Year = 2018, 2511/3075, Rosario, Randy
Year = 2018, 2512/3075, Bergman, Christian
Year = 2018, 2513/3075, Baumann, Buddy
Year = 2018, 2514/3075, Neris, Héctor
Year = 2018, 2515/3075, Turner, Jacob
Year = 2018, 2516/3075, Barbato, Johnny
Year = 2018, 2517/3075, Hicks, Jordan
Year = 2018, 2518/3075, Melancon, Mark
Year = 2018, 2519/3075, Rainey, Tanner
Year = 2018, 2520/3075, Deetz, Dean
Year = 2018, 2521/3075, Bleier, Richard
Year = 2018, 2522/3075, Montas, Frankie
Year = 2018, 2523/3075, Hirano, Yoshihisa
Year = 2018, 2524/3075, Wright, Steven
Year = 2018, 2525/3075, Zagurski, Mike
Year = 2018, 2526/3075, Norris, Daniel
Year = 2018, 2527/3075, Okert, Steven
Year = 2018, 2528/3075, Duke, Zach
Year = 2018, 2529/3075, Rosscup, Z

Year = 2018, 2728/3075, Sampson, Adrian
Year = 2018, 2729/3075, Banda, Anthony
Year = 2018, 2730/3075, Anderson, Tanner
Year = 2018, 2731/3075, McCullers Jr., Lance
Year = 2018, 2732/3075, Sadler, Casey
Year = 2018, 2733/3075, Hoyt, James
Year = 2018, 2734/3075, Davis, J.D.
Year = 2018, 2735/3075, Smolinski, Jake
Year = 2018, 2736/3075, Holmes, Clay
Year = 2018, 2737/3075, Fried, Max
Year = 2018, 2738/3075, Shafer, Justin
Year = 2018, 2739/3075, LaMarre, Ryan
Year = 2018, 2740/3075, Smith, Murphy
Year = 2018, 2741/3075, Milner, Hoby
Year = 2018, 2742/3075, Vázquez, Felipe
Year = 2018, 2743/3075, Sucre, Jesús
Year = 2018, 2744/3075, Ramírez, Yefry
Year = 2018, 2745/3075, Whitley, Chase
Year = 2018, 2746/3075, McCreery, Adam
Year = 2018, 2747/3075, Ortiz, Luis F.
Year = 2018, 2748/3075, Gómez, Carlos
Year = 2018, 2749/3075, Bleier, Richard
Year = 2018, 2750/3075, Parsons, Wes
Year = 2018, 2751/3075, Alvarado, José
Year = 2018, 2752/3075, Mariñez, Jhan
Year = 2018, 2753/3075, Peterson, Ja

Year = 2018, 2943/3075, Oswalt, Corey
Year = 2018, 2944/3075, Edwards, Jon
Year = 2018, 2945/3075, Ogando, Alexi
Year = 2018, 2946/3075, Wisler, Matt
Year = 2018, 2947/3075, Mayers, Mike
Year = 2018, 2948/3075, Lyons, Tyler
Year = 2018, 2949/3075, Hanhold, Eric
Year = 2018, 2950/3075, Urías, Julio
Year = 2018, 2951/3075, García, Yimi
Year = 2018, 2952/3075, Brebbia, John
Year = 2018, 2953/3075, McGuire, Deck
Year = 2018, 2954/3075, Rua, Ryan
Year = 2018, 2955/3075, Jewell, Jake
Year = 2018, 2956/3075, Hill, Rich
Year = 2018, 2957/3075, Taylor, Ben
Year = 2018, 2958/3075, Beliveau, Jeff
Year = 2018, 2959/3075, Weaver, Luke
Year = 2018, 2960/3075, Gyorko, Jedd
Year = 2018, 2961/3075, Valencia, Danny
Year = 2018, 2962/3075, Hill, Rich
Year = 2018, 2963/3075, Webster, Allen
Year = 2018, 2964/3075, Gohara, Luiz
Year = 2018, 2965/3075, Hudson, Daniel
Year = 2018, 2966/3075, Copeland, Scott
Year = 2018, 2967/3075, Betances, Dellin
Year = 2018, 2968/3075, Goeddel, Erik
Year = 2018, 2969/3075, 